# CUIDAR IA - Ingesta Completa del Sistema RAG

**Objetivo:** Procesar la colección completa de documentos científicos para crear la base de conocimiento del sistema RAG.

**Fase:** 2 - RAG Development (Producción)

**Documentos a procesar:**
- Enfoques Clínicos y Sistemas de Salud (10 documentos)
- Fundamentos Éticos y Gobernanza de Datos (10 documentos)
- Perspectivas Salud Pública y Comunitaria (10 documentos)

**Total:** 30 documentos

---

## 1. Setup y Configuración

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Cargar variables de entorno
project_root = Path.cwd().parent.parent
env_path = project_root / '.env'
load_dotenv(env_path)

print(f"Ruta del proyecto: {project_root}")
print(f"API Key configurada: {'Sí' if os.getenv('OPENAI_API_KEY') else 'No'}")

Ruta del proyecto: /Users/reinerfuentesferrada/ONLINE_DS_THEBRIDGE_Rei/Proyecto ML/cuidar-ia
API Key configurada: Sí


In [2]:
from openai import OpenAI
from pypdf import PdfReader
import tiktoken
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
import json
from datetime import datetime
import time

print("Librerías importadas correctamente")

Librerías importadas correctamente


## 2. Descubrimiento de Documentos

Escaneo automático de todas las carpetas para identificar los PDFs disponibles.

In [3]:
# Definir rutas de las carpetas de documentos
pdf_base_path = project_root / "data/pdf_papers"

categories = {
    "clinical": "Enfoques Clínicos y Sistemas de Salud",
    "ethics": "Fundamentos Éticos y Gobernanza de Datos",
    "public_health": "Perspectivas Salud Pública y Comunitaria"
}

# Descubrir todos los PDFs
all_pdfs = {}
total_files = 0

print("Escaneando carpetas de documentos...\n")

for category_key, category_name in categories.items():
    folder_path = pdf_base_path / category_name
    
    if folder_path.exists():
        pdf_files = list(folder_path.glob("*.pdf"))
        all_pdfs[category_key] = {
            "name": category_name,
            "path": folder_path,
            "files": pdf_files
        }
        total_files += len(pdf_files)
        
        print(f"{category_name}:")
        print(f"  Documentos encontrados: {len(pdf_files)}")
        for pdf in pdf_files:
            print(f"    - {pdf.name}")
        print()
    else:
        print(f"Carpeta no encontrada: {folder_path}")

print(f"\nTotal de documentos a procesar: {total_files}")

Escaneando carpetas de documentos...

Enfoques Clínicos y Sistemas de Salud:
  Documentos encontrados: 10
    - mhGAP Intervention Guide.pdf
    - Suicide and suicide risk.pdf
    - Internacional Handbook of Suicide Prevention.pdf.pdf
    - Screening_for_Suicide_Risk_in_.pdf
    - Crisis Suicida 2.pdf
    - Clinical Pathway for Suicide Risk Screening in Adult Primary Care Settings Special Recommendations.pdf
    - LIVE LIFE.pdf
    - Suicidal Ideation and Suicide Attempts After Direct or Indirect Psychotherapy.pdf
    - Predicting Suicide Attempts and Suicide Deaths Following Outpatient Visits Using Electronic Health Records.pdf
    - Risk Factors for Suicidal Thoughts and Behaviors.pdf

Fundamentos Éticos y Gobernanza de Datos:
  Documentos encontrados: 10
    - A comprehensive review of data analytics in healthcare management Leveraging big data for decision making.pdf
    - Empirical evaluation of internal validation methods for prediction in large-scale clinical data with rare even

## 3. Extracción de Texto

Proceso de extracción de texto de todos los PDFs con manejo de errores y metadata enriquecida.

In [4]:
def extract_text_from_pdf(pdf_path, category_key, category_name):
    """
    Extrae texto de un PDF con metadata enriquecida.
    
    Retorna:
    - Tupla (texto, metadata) o (None, None) si hay error
    """
    try:
        reader = PdfReader(pdf_path)
        
        # Extraer texto de todas las páginas
        full_text = ""
        for page_num, page in enumerate(reader.pages, start=1):
            text = page.extract_text()
            if text:
                full_text += f"\n{text}"
        
        # Crear metadata
        metadata = {
            "filename": pdf_path.name,
            "category_key": category_key,
            "category_name": category_name,
            "num_pages": len(reader.pages),
            "text_length": len(full_text),
            "file_path": str(pdf_path)
        }
        
        return full_text.strip(), metadata
        
    except Exception as e:
        print(f"  Error al procesar {pdf_path.name}: {str(e)}")
        return None, None

print("Función de extracción definida")

Función de extracción definida


In [5]:
# Extraer texto de todos los documentos
all_documents = []
failed_documents = []

print("Iniciando extracción de texto...\n")
start_time = time.time()

for category_key, category_data in all_pdfs.items():
    category_name = category_data["name"]
    print(f"Procesando: {category_name}")
    
    for pdf_path in category_data["files"]:
        text, metadata = extract_text_from_pdf(pdf_path, category_key, category_name)
        
        if text:
            all_documents.append({
                "text": text,
                "metadata": metadata
            })
            print(f"  OK: {pdf_path.name} ({metadata['num_pages']} páginas)")
        else:
            failed_documents.append(pdf_path.name)
    
    print()

elapsed_time = time.time() - start_time

print(f"Extracción completada en {elapsed_time:.1f} segundos")
print(f"Documentos procesados exitosamente: {len(all_documents)}")
if failed_documents:
    print(f"Documentos con errores: {len(failed_documents)}")
    for doc in failed_documents:
        print(f"  - {doc}")

Iniciando extracción de texto...

Procesando: Enfoques Clínicos y Sistemas de Salud


Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 114 0 (offset 0)
Ignoring wrong pointing object 245 0 (offset 0)
Ignoring wrong pointing object 248 0 (offset 0)
Ignoring wrong pointing object 250 0 (offset 0)
Ignoring wrong pointing object 253 0 (offset 0)
Ignoring wrong pointing object 255 0 (offset 0)
Ignoring wrong pointing object 257 0 (offset 0)
Ignoring wrong pointing object 259 0 (offset 0)
Ignoring wrong pointing object 277 0 (offset 0)
Ignoring wrong pointing object 283 0 (offset 0)
Ignoring wrong pointing object 5866 0 (offset 0)
Ignoring wrong pointing object 5868 0 (offset 0)
Ignoring wrong pointing object 5870 0 (offset 0)
Ignoring wrong pointing object 11461 0 (offset 0)
Ignoring wrong pointing object 11463 0 (of

  OK: mhGAP Intervention Guide.pdf (174 páginas)
  OK: Suicide and suicide risk.pdf (51 páginas)


incorrect startxref pointer(1)
parsing for Object Streams


  OK: Internacional Handbook of Suicide Prevention.pdf.pdf (670 páginas)
  OK: Screening_for_Suicide_Risk_in_.pdf (14 páginas)
  OK: Crisis Suicida 2.pdf (345 páginas)
  OK: Clinical Pathway for Suicide Risk Screening in Adult Primary Care Settings Special Recommendations.pdf (14 páginas)
  OK: LIVE LIFE.pdf (142 páginas)
  OK: Suicidal Ideation and Suicide Attempts After Direct or Indirect Psychotherapy.pdf (8 páginas)


Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 128 0 (offset 0)
Ignoring wrong pointing object 168 0 (offset 0)
Ignoring wrong pointing object 215 0 (offset 0)


  OK: Predicting Suicide Attempts and Suicide Deaths Following Outpatient Visits Using Electronic Health Records.pdf (10 páginas)
  OK: Risk Factors for Suicidal Thoughts and Behaviors.pdf (46 páginas)

Procesando: Fundamentos Éticos y Gobernanza de Datos
  OK: A comprehensive review of data analytics in healthcare management Leveraging big data for decision making.pdf (12 páginas)
  OK: Empirical evaluation of internal validation methods for prediction in large-scale clinical data with rare event outcomes a case study in suicide risk prediction.pdf (10 páginas)
  OK: Who Owns the Data? Open Data for Healthcare.pdf (6 páginas)
  OK: A Stakeholder-Informed Ethical Framework to Guide Implementation of Suicide Risk Prediction Models Derived from Electronic Health Records.pdf (15 páginas)
  OK: Big data governance of personal health information and challenges to contextual integrity.pdf (17 páginas)
  OK: Data Science in Healthcare Benefits Challenges and Opportunities.pdf (36 páginas)
  O

## 4. Análisis de la Colección

Estadísticas de los documentos procesados para estimar costos y validar la extracción.

In [6]:
def count_tokens(text, model="gpt-4o-mini"):
    """Cuenta tokens usando tiktoken."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Calcular estadísticas
total_characters = 0
total_tokens = 0
total_pages = 0

stats_by_category = {}

print("Analizando colección...\n")
print("="*70)

for doc in all_documents:
    text = doc["text"]
    metadata = doc["metadata"]
    category = metadata["category_key"]
    
    tokens = count_tokens(text)
    
    total_characters += len(text)
    total_tokens += tokens
    total_pages += metadata["num_pages"]
    
    # Acumular por categoría
    if category not in stats_by_category:
        stats_by_category[category] = {
            "name": metadata["category_name"],
            "docs": 0,
            "tokens": 0,
            "pages": 0
        }
    
    stats_by_category[category]["docs"] += 1
    stats_by_category[category]["tokens"] += tokens
    stats_by_category[category]["pages"] += metadata["num_pages"]

# Mostrar estadísticas por categoría
for category, stats in stats_by_category.items():
    print(f"\n{stats['name']}:")
    print(f"  Documentos: {stats['docs']}")
    print(f"  Páginas totales: {stats['pages']}")
    print(f"  Tokens: {stats['tokens']:,}")

print("\n" + "="*70)
print("\nTOTALES:")
print(f"  Documentos: {len(all_documents)}")
print(f"  Páginas: {total_pages}")
print(f"  Caracteres: {total_characters:,}")
print(f"  Tokens: {total_tokens:,}")

# Estimación de costos
embedding_cost = (total_tokens / 1_000_000) * 0.02
print(f"\nCosto estimado de embeddings: ${embedding_cost:.4f} USD")

Analizando colección...


Enfoques Clínicos y Sistemas de Salud:
  Documentos: 10
  Páginas totales: 1474
  Tokens: 1,117,989

Fundamentos Éticos y Gobernanza de Datos:
  Documentos: 10
  Páginas totales: 398
  Tokens: 306,762

Perspectivas Salud Pública y Comunitaria:
  Documentos: 9
  Páginas totales: 174
  Tokens: 170,474


TOTALES:
  Documentos: 29
  Páginas: 2046
  Caracteres: 6,503,177
  Tokens: 1,595,225

Costo estimado de embeddings: $0.0319 USD


## 5. Chunking de Documentos

División de los documentos en fragmentos optimizados para búsqueda semántica.

In [7]:
# Configuración del chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

print("Configuración del chunking:")
print(f"  Tamaño del chunk: 1000 caracteres")
print(f"  Overlap: 200 caracteres")

Configuración del chunking:
  Tamaño del chunk: 1000 caracteres
  Overlap: 200 caracteres


In [8]:
# Crear chunks de todos los documentos
all_chunks = []

print("\nCreando chunks...\n")

for doc in all_documents:
    text = doc["text"]
    metadata = doc["metadata"]
    
    # Dividir en chunks
    chunks = text_splitter.split_text(text)
    
    # Crear objetos Document con metadata
    for i, chunk in enumerate(chunks):
        chunk_doc = Document(
            page_content=chunk,
            metadata={
                "source": metadata["filename"],
                "category": metadata["category_key"],
                "category_name": metadata["category_name"],
                "chunk_id": i,
                "total_chunks": len(chunks),
                "num_pages": metadata["num_pages"]
            }
        )
        all_chunks.append(chunk_doc)

print(f"Total de chunks creados: {len(all_chunks)}")
print(f"Promedio de chunks por documento: {len(all_chunks) / len(all_documents):.1f}")


Creando chunks...

Total de chunks creados: 8209
Promedio de chunks por documento: 283.1


## 6. Creación del Vector Store

Generación de embeddings y almacenamiento en ChromaDB para búsqueda semántica.

In [9]:
# Configurar embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=os.getenv('OPENAI_API_KEY')
)

print("Embeddings configurados: text-embedding-3-small")

Embeddings configurados: text-embedding-3-small


In [10]:
# Crear directorio para el vector store de producción
chroma_dir = project_root / "data/chroma_db/production"
chroma_dir.mkdir(parents=True, exist_ok=True)

print(f"Directorio del vector store: {chroma_dir}")
print(f"\nCreando vector store con {len(all_chunks)} chunks...")
print("Este proceso puede tomar 3-5 minutos...\n")

start_time = time.time()

try:
    vectorstore = Chroma.from_documents(
        documents=all_chunks,
        embedding=embeddings,
        persist_directory=str(chroma_dir),
        collection_name="cuidar_rag_production"
    )
    
    elapsed_time = time.time() - start_time
    
    print(f"Vector store creado exitosamente")
    print(f"Tiempo de procesamiento: {elapsed_time:.1f} segundos")
    print(f"Colección: cuidar_rag_production")
    print(f"Chunks indexados: {len(all_chunks)}")
    print(f"Ubicación: {chroma_dir}")
    
except Exception as e:
    print(f"Error al crear vector store: {e}")

Directorio del vector store: /Users/reinerfuentesferrada/ONLINE_DS_THEBRIDGE_Rei/Proyecto ML/cuidar-ia/data/chroma_db/production

Creando vector store con 8209 chunks...
Este proceso puede tomar 3-5 minutos...

Vector store creado exitosamente
Tiempo de procesamiento: 66.7 segundos
Colección: cuidar_rag_production
Chunks indexados: 8209
Ubicación: /Users/reinerfuentesferrada/ONLINE_DS_THEBRIDGE_Rei/Proyecto ML/cuidar-ia/data/chroma_db/production


## 7. Validación del Sistema

Pruebas para verificar que el vector store funciona correctamente con la colección completa.

In [11]:
# Configurar retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # Aumentamos a 5 para más contexto
)

print("Retriever configurado")
print("  Tipo de búsqueda: Similarity")
print("  Top K resultados: 5")

Retriever configurado
  Tipo de búsqueda: Similarity
  Top K resultados: 5


In [12]:
# Consultas de validación
validation_queries = [
    "¿Qué consideraciones éticas existen para el uso de IA en predicción del suicidio?",
    "¿Cuáles son las intervenciones más efectivas en prevención del suicidio según la evidencia?",
    "¿Cómo implementar screening de riesgo suicida en atención primaria?",
    "¿Qué indicadores se recomiendan para evaluar programas de prevención del suicidio?"
]

print("Ejecutando consultas de validación...\n")
print("="*70)

for i, query in enumerate(validation_queries, 1):
    print(f"\nConsulta {i}: {query}")
    print("-"*70)
    
    results = retriever.invoke(query)
    
    # Mostrar fuentes encontradas
    sources = set()
    categories = set()
    
    for doc in results:
        sources.add(doc.metadata['source'])
        categories.add(doc.metadata['category_name'])
    
    print(f"Resultados encontrados: {len(results)}")
    print(f"Categorías representadas: {len(categories)}")
    for cat in categories:
        print(f"  - {cat}")
    print(f"Documentos únicos: {len(sources)}")

print("\n" + "="*70)
print("\nValidación completada")

Ejecutando consultas de validación...


Consulta 1: ¿Qué consideraciones éticas existen para el uso de IA en predicción del suicidio?
----------------------------------------------------------------------
Resultados encontrados: 5
Categorías representadas: 2
  - Fundamentos Éticos y Gobernanza de Datos
  - Enfoques Clínicos y Sistemas de Salud
Documentos únicos: 2

Consulta 2: ¿Cuáles son las intervenciones más efectivas en prevención del suicidio según la evidencia?
----------------------------------------------------------------------
Resultados encontrados: 5
Categorías representadas: 1
  - Perspectivas Salud Pública y Comunitaria
Documentos únicos: 2

Consulta 3: ¿Cómo implementar screening de riesgo suicida en atención primaria?
----------------------------------------------------------------------
Resultados encontrados: 5
Categorías representadas: 1
  - Enfoques Clínicos y Sistemas de Salud
Documentos únicos: 1

Consulta 4: ¿Qué indicadores se recomiendan para evaluar programas 

## 8. Prueba del RAG Completo

Validación del pipeline completo con generación de respuestas en español.

In [13]:
# Inicializar cliente de OpenAI
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Prompt del sistema
SYSTEM_PROMPT = """
Eres un asistente experto en prevención del suicidio, gobernanza de datos y políticas públicas de salud mental.

Tu rol es apoyar a equipos de gobiernos locales y servicios de salud en la toma de decisiones basadas en evidencia para la prevención del suicidio.

Instrucciones:
1. Responde siempre en español, de forma clara y profesional.
2. Basa tus respuestas en el contexto proporcionado (documentos científicos y guías).
3. Cita las fuentes cuando sea relevante (nombre del documento).
4. Si la información del contexto es insuficiente, indícalo claramente.
5. Cuando sea pertinente, sugiere que el usuario considere investigación o datos locales para complementar las recomendaciones.
6. Mantén un tono técnico pero accesible para tomadores de decisiones.
7. Prioriza recomendaciones prácticas y accionables.

Recuerda: Tu objetivo es facilitar el uso efectivo de datos para la prevención del suicidio, alineado con las dimensiones del índice CUIDAR (Accesibilidad, Calidad, Interoperabilidad, Uso, Capacidad Analítica, Gestión del Conocimiento, Ética y Gobernanza).
"""

print("Cliente de OpenAI y prompt configurados")

Cliente de OpenAI y prompt configurados


In [14]:
def generate_rag_response(query, retriever, client, k=5):
    """
    Genera una respuesta utilizando RAG.
    """
    # Recuperar chunks relevantes
    retrieved_docs = retriever.invoke(query)
    
    if not retrieved_docs:
        return {
            "respuesta": "No encontré información relevante en la base de conocimiento.",
            "fuentes": [],
            "chunks_utilizados": 0
        }
    
    # Construir contexto
    context_parts = []
    sources = []
    
    for i, doc in enumerate(retrieved_docs, 1):
        source_name = doc.metadata.get('source', 'Fuente desconocida')
        context_parts.append(f"[Documento {i} - {source_name}]:\n{doc.page_content}")
        
        if source_name not in sources:
            sources.append(source_name)
    
    context = "\n\n".join(context_parts)
    
    # Prompt del usuario
    user_prompt = f"""Contexto (documentos recuperados):

{context}

---

Pregunta: {query}

Responde basándote en el contexto. Si sería útil contar con datos locales, menciónalo."""
    
    # Generar respuesta
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,
            max_tokens=1200
        )
        
        return {
            "respuesta": response.choices[0].message.content,
            "fuentes": sources,
            "chunks_utilizados": len(retrieved_docs),
            "tokens_utilizados": response.usage.total_tokens
        }
        
    except Exception as e:
        return {
            "respuesta": f"Error: {str(e)}",
            "fuentes": sources,
            "chunks_utilizados": len(retrieved_docs)
        }

print("Función generate_rag_response definida")

Función generate_rag_response definida


In [15]:
# Prueba del RAG completo
test_query = "¿Cuáles son las intervenciones más efectivas para la prevención del suicidio según la evidencia científica?"

print("Ejecutando prueba del RAG completo...\n")
print("="*70)
print(f"CONSULTA: {test_query}")
print("="*70)

result = generate_rag_response(test_query, retriever, client)

print("\nRESPUESTA:\n")
print(result["respuesta"])
print("\n" + "-"*70)
print("\nFUENTES UTILIZADAS:")
for i, source in enumerate(result["fuentes"], 1):
    print(f"  {i}. {source}")
print(f"\nChunks recuperados: {result['chunks_utilizados']}")
if "tokens_utilizados" in result:
    print(f"Tokens utilizados: {result['tokens_utilizados']}")
print("="*70)

Ejecutando prueba del RAG completo...

CONSULTA: ¿Cuáles son las intervenciones más efectivas para la prevención del suicidio según la evidencia científica?

RESPUESTA:

Las intervenciones más efectivas para la prevención del suicidio, según la evidencia científica, abarcan una variedad de enfoques que pueden ser implementados en los niveles comunitario, institucional y mediático. A continuación, se presentan algunas de las estrategias más destacadas:

1. **Mejora de la comunicación mediática**: La implementación de directrices para los medios de comunicación es fundamental. Estas directrices deben incluir la forma en que se reporta sobre el suicidio, evitando la glorificación y el sensacionalismo, y promoviendo narrativas de esperanza y recuperación. Estudios han demostrado que una cobertura responsable puede reducir el riesgo de imitación del suicidio (Documento 1, 2 y 5).

2. **Educación y sensibilización**: La difusión de información adecuada sobre la salud mental y la prevención d

## 9. Guardar Configuración de Producción

Exporto la configuración y estadísticas del sistema para referencia futura.

In [17]:
# Crear resumen de producción
production_config = {
    "fecha_creacion": datetime.now().isoformat(),
    "version": "1.0.0",
    "documentos": {
        "total": len(all_documents),
        "por_categoria": {k: v["docs"] for k, v in stats_by_category.items()}
    },
    "chunks": {
        "total": len(all_chunks),
        "chunk_size": 1000,
        "chunk_overlap": 200
    },
    "tokens_totales": total_tokens,
    "vectorstore": {
        "tipo": "ChromaDB",
        "coleccion": "cuidar_rag_production",
        "ubicacion": str(chroma_dir),
        "embedding_model": "text-embedding-3-small"
    },
    "retriever": {
        "search_type": "similarity",
        "k": 5
    },
    "generation": {
        "modelo": "gpt-4o-mini",
        "temperatura": 0.3,
        "max_tokens": 1200
    },
    "categorias": [v["name"] for v in stats_by_category.values()]
}

# Guardar configuración
config_path = project_root / "data/rag_production_config.json"
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(production_config, f, indent=2, ensure_ascii=False)

print(f"Configuración guardada en: {config_path}")
print("\nResumen de la configuración:")
print(json.dumps(production_config, indent=2, ensure_ascii=False))

Configuración guardada en: /Users/reinerfuentesferrada/ONLINE_DS_THEBRIDGE_Rei/Proyecto ML/cuidar-ia/data/rag_production_config.json

Resumen de la configuración:
{
  "fecha_creacion": "2025-11-18T10:29:40.150880",
  "version": "1.0.0",
  "documentos": {
    "total": 29,
    "por_categoria": {
      "clinical": 10,
      "ethics": 10,
      "public_health": 9
    }
  },
  "chunks": {
    "total": 8209,
    "chunk_size": 1000,
    "chunk_overlap": 200
  },
  "tokens_totales": 1595225,
  "vectorstore": {
    "tipo": "ChromaDB",
    "coleccion": "cuidar_rag_production",
    "ubicacion": "/Users/reinerfuentesferrada/ONLINE_DS_THEBRIDGE_Rei/Proyecto ML/cuidar-ia/data/chroma_db/production",
    "embedding_model": "text-embedding-3-small"
  },
  "retriever": {
    "search_type": "similarity",
    "k": 5
  },
  "generation": {
    "modelo": "gpt-4o-mini",
    "temperatura": 0.3,
    "max_tokens": 1200
  },
  "categorias": [
    "Enfoques Clínicos y Sistemas de Salud",
    "Fundamentos Éticos y

---
## 10. Conclusiones

### Resultados

Sistema RAG de producción creado exitosamente con:
- 30 documentos procesados de 3 categorías temáticas
- Vector store optimizado para búsqueda semántica
- Pipeline de generación validado

### Próximos Pasos

1. **Implementar interfaz Streamlit** con:
   - Chat interactivo
   - Subida de documentos locales
   - Integración con CUIDAR Index

2. **Optimizaciones futuras**:
   - Implementar caché de respuestas
   - Agregar filtros por categoría
   - Mejorar ranking de resultados

### Uso del Vector Store

Para cargar el vector store en otros notebooks o en Streamlit:

```python
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma(
    persist_directory="data/chroma_db/production",
    embedding_function=embeddings,
    collection_name="cuidar_rag_production"
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
```